# Postgres Vector Store
In this notebook we are going to show how to use [Postgresql](https://www.postgresql.org) and  [pgvector](https://github.com/pgvector/pgvector)  to perform vector searches in LlamaIndex

In [10]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import PGVectorStore
import textwrap
import openai

### Setup OpenAI
The first step is to configure the openai key. It will be used to created embeddings for the documents loaded into the index

In [11]:
import os
os.environ['OPENAI_API_KEY'] = "<your key>"
openai.api_key = "<your key>"

### Loading documents
Load the documents stored in the `paul_graham_essay` using the SimpleDirectoryReader

In [12]:
documents = SimpleDirectoryReader('../data/paul_graham').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

Document ID: 4842475e-cf28-4b77-885d-9c2dfb5658f4 Document Hash: 77ae91ab542f3abb308c4d7c77c9bc4c9ad0ccd63144802b7cbe7e1bb3a4094e


### Create the index
Here we create an index backed by Postgres using the documents loaded previously. PGVectorStore takes a few arguments.

In [13]:
vector_store = PGVectorStore.from_params(
            database="vector_db",
            host="localhost",
            password="",
            port=5432,
            user="postgres",
            table_name="paul_graham_essay"
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the index
We can now ask questions using our index.

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")

In [15]:
print(textwrap.fill(str(response), 100))

 The author is Paul Graham.


In [16]:
response = query_engine.query("What did the author do growing up?")

In [17]:
print(textwrap.fill(str(response), 100))

 The author grew up writing essays on topics they had stacked up, thinking about what to work on,
and exploring the implications of the internet. They also studied Italian, explored Florence,
painted people and still lifes, and worked on software projects. They wrote essays for others to
read and published them online. They also had dinners with friends and bought a building in
Cambridge.


### Querying existing index

In [19]:
from llama_index import load_index_from_storage

vector_store = PGVectorStore.from_params(
            database="vector_db",
            host="localhost",
            password="",
            port=5432,
            user="postgres",
            table_name="paul_graham_essay"
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")

In [20]:
print(textwrap.fill(str(response), 100))


 The author is Paul Graham.
